<a href="https://colab.research.google.com/github/krishnanravi/llm-experiments/blob/main/1.%20transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q transformers accelerate bitsandbytes

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
OPT = AutoModelForCausalLM.from_pretrained("facebook/opt-1.3b")

pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [ ]:
inp = "The quick brown fox jumps over the lazy dog"
inp_tokenized = tokenizer( inp, return_tensors="pt" )
print( inp_tokenized['input_ids'].size() )
print( inp_tokenized )

torch.Size([1, 10])
{'input_ids': tensor([[    2,   133,  2119,  6219, 23602, 13855,    81,     5, 22414,  2335]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
print( OPT.model )

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 2048, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
    (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTSdpaAttention(
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      )


In [ ]:
embedded_input = OPT.model.decoder.embed_tokens(inp_tokenized['input_ids'])
print( "Layer:\t", OPT.model.decoder.embed_tokens )
print( "Size:\t", embedded_input.size() )
print( "Output:\t", embedded_input )

Layer:	 Embedding(50272, 2048, padding_idx=1)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0407,  0.0519,  0.0574,  ..., -0.0263, -0.0355, -0.0260],
         [-0.0371,  0.0220, -0.0096,  ...,  0.0265, -0.0166, -0.0030],
         [-0.0455, -0.0236, -0.0121,  ...,  0.0043, -0.0166,  0.0193],
         ...,
         [ 0.0007,  0.0267,  0.0257,  ...,  0.0622,  0.0421,  0.0279],
         [-0.0126,  0.0347, -0.0352,  ..., -0.0393, -0.0396, -0.0102],
         [-0.0115,  0.0319,  0.0274,  ..., -0.0472, -0.0059,  0.0341]]],
       grad_fn=<EmbeddingBackward0>)


In [ ]:
embed_pos_input = OPT.model.decoder.embed_positions(inp_tokenized['attention_mask'])
print( "Layer:\t", OPT.model.decoder.embed_positions )
print( "Size:\t", embed_pos_input.size() )
print( "Output:\t", embed_pos_input )

Layer:	 OPTLearnedPositionalEmbedding(2050, 2048)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-8.1406e-03, -2.6221e-01,  6.0768e-03,  ...,  1.7273e-02,
          -5.0621e-03, -1.6220e-02],
         [-8.0585e-05,  2.5000e-01, -1.6632e-02,  ..., -1.5419e-02,
          -1.7838e-02,  2.4948e-02],
         [-9.9411e-03, -1.4978e-01,  1.7557e-03,  ...,  3.7117e-03,
          -1.6434e-02, -9.9087e-04],
         ...,
         [ 3.6979e-04, -7.7454e-02,  1.2955e-02,  ...,  3.9330e-03,
          -1.1642e-02,  7.8506e-03],
         [-2.6779e-03, -2.2446e-02, -1.6754e-02,  ..., -1.3142e-03,
          -7.8583e-03,  2.0096e-02],
         [-8.6288e-03,  1.4233e-01, -1.9012e-02,  ..., -1.8463e-02,
          -9.8572e-03,  8.7662e-03]]], grad_fn=<EmbeddingBackward0>)


In [ ]:
embed_position_input = embedded_input + embed_pos_input
hidden_states, _, _ = OPT.model.decoder.layers[0].self_attn( embed_position_input )
print( "Layer:\t", OPT.model.decoder.layers[0].self_attn )
print( "Size:\t", hidden_states.size() )
print( "Output:\t", hidden_states )

Layer:	 OPTSdpaAttention(
  (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
  (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
  (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
  (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
)
Size:	 torch.Size([1, 10, 2048])
Output:	 tensor([[[-0.0135, -0.0095,  0.0013,  ...,  0.0065, -0.0017,  0.0134],
         [-0.0130, -0.0102,  0.0022,  ...,  0.0087,  0.0004,  0.0124],
         [-0.0131, -0.0059,  0.0038,  ...,  0.0098,  0.0020,  0.0141],
         ...,
         [-0.0121, -0.0098,  0.0050,  ...,  0.0095,  0.0015,  0.0098],
         [-0.0119, -0.0101,  0.0051,  ...,  0.0094,  0.0011,  0.0091],
         [-0.0118, -0.0109,  0.0055,  ...,  0.0095,  0.0013,  0.0091]]],
       grad_fn=<ViewBackward0>)


In [7]:
!pip install -q transformers accelerate bitsandbytes

from transformers import AutoModel, AutoTokenizer

BART = AutoModel.from_pretrained("facebook/bart-large")
print(BART)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.0 MB/s eta 0:00:00


config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

BartModel(
  (shared): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
  (encoder): BartEncoder(
    (embed_tokens): BartScaledWordEmbedding(50265, 1024, padding_idx=1)
    (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
    (layers): ModuleList(
      (0-11): 12 x BartEncoderLayer(
        (self_attn): BartSdpaAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=

In [8]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

Device set to use cpu


In [12]:
sum = summarizer("""A fundamental distinction in the encoder-only architecture is the absence of a masked self-attention layer used in decoder models to prevent future tokens from influencing the current token during training. In encoder-only models, the self-attention mechanism processes the entire input sequence simultaneously, allowing the model to capture the full context. This characteristic makes them exceptionally well-suited for generating comprehensive vector representations of documents, ensuring that all information is retained.""", min_length=10, max_length=15)
print(sum[0]['summary_text'])

In encoder-only models, the self-attention


In [13]:
BERT = AutoModel.from_pretrained("bert-base-uncased")
print(BERT)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [19]:
classifier = pipeline("text-classification",
model="nlptown/bert-base-multilingual-uncased-sentiment")
lbl = classifier("""This restaurant is awesomely awful.""")

print(lbl)

Device set to use cpu


[{'label': '1 star', 'score': 0.9050528407096863}]


In [20]:
gpt2 = AutoModel.from_pretrained("gpt2")
print(gpt2)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2SdpaAttention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)


In [21]:
generator = pipeline(model="gpt2")
output = generator("This movie was a very", do_sample=True,
top_p=0.95, num_return_sequences=4, max_new_tokens=50, return_full_text=False)

for item in output:
  print(">", item['generated_text'])

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


>  good time and I'm really proud of myself for not having too many people that are going to get to see the movie so I can take part in the event."

To mark the announcement, the studio will launch a new social app for the
>  big success, as did most of the other great sequels (Star Trek, Alien, etc) and is now being reviewed with great acclaim by many fans around the globe.

(If you are a Star Wars fan I would highly recommend watching this
>  interesting one for me. Not just because I was the first to admit that I loved it, and I didn't know how to handle it. I love that movie, so if I had to describe it to someone I didn't know, maybe I
>  interesting one and I love it. He (Teddy Bear) really got me excited and it's about his love of his dogs. He's the one that makes me come back and make the best of what I'm about to say. It's


In [14]:
from transformers import AutoModel, AutoTokenizer

deepseek = AutoModel.from_pretrained("deepseek-ai/DeepSeek-R1", trust_remote_code=True, device_map="auto", load_in_8bit=True)
print(deepseek)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


ValueError: Unknown quantization type, got fp8 - supported types are: ['awq', 'bitsandbytes_4bit', 'bitsandbytes_8bit', 'gptq', 'aqlm', 'quanto', 'eetq', 'hqq', 'compressed-tensors', 'fbgemm_fp8', 'torchao', 'bitnet']